[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1dDaaTAipo6C-Z-lkf-mJvHzG0z8G6gjL#scrollTo=lWOsAvobfEdK)

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=af7e68d4a4cab1b312f4e60afa1909f55571a4b658b10aaa983ac42fa6bad513
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import torch
from torchvision import datasets, transforms
from tqdm import tqdm
from torchvision.transforms import v2
from torchvision.transforms import Lambda
# Define the transform to convert images to grayscale
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((28, 28), antialias=True),
    v2.Grayscale(),
    Lambda(lambda x: torch.flatten(x))
])

# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Placeholder for the sum and squared sum of the images (used for mean and std calculation)
sum_intensity = 0.0
sum_squared_intensity = 0.0
num_elements = 0

# Loop over the dataset
for data, _ in tqdm(train_dataset, desc='Computing mean and std'):
    sum_intensity += torch.mean(data)
    sum_squared_intensity += torch.mean(data**2)
    num_elements += 1

# Calculate mean and standard deviation
mean = sum_intensity / num_elements
std = (sum_squared_intensity / num_elements - mean ** 2) ** 0.5

print(f'Mean: {mean}')
print(f'Std: {std}')


100%|██████████| 170498071/170498071 [00:10<00:00, 16396738.04it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


Computing mean and std: 100%|██████████| 50000/50000 [00:43<00:00, 1145.02it/s]


Mean: 0.48075222969055176
Std: 0.23171930015087128


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.transforms import v2
from torchvision.transforms import Lambda
import wandb

# Sweep Configuration
sweep_config = {
    'name': 'my-sweep3',
    'method': 'random',  # can be grid, random, or bayes
    'metric': {
      'goal': 'maximize',
      'name': 'epoch_validation_accuracy'
    },
    'parameters': {
        'optimizer': {
            'values': ['sgd','adam', 'rmsprop','adagrad','sgd_sam']
        },
        'learning_rate': {
            'values': [1e-3, 1e-4] # 1e-5
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'weight_decay': {
            'values': [0, 1e-4, 1e-2]  # Example values; 1e-3
        },
        'momentum': {  # Only relevant for SGD and SGD_SAM
            'values': [0.9, 0.99] #  0.95
        },
        # 'nesterov': {
        #     'values': [False, True]  # Typically used for SGD
        # },
        # 'rho': {  # SAM-specific hyperparameter
        #     'values': [0.05, 0.1, 0.2]
        # }
        # ... add other parameters as needed
    }
}



# Your MLP and accuracy functions remain the same
class MLP(nn.Module):
    def __init__(self, input_dim=784, hidden_layers=[1024, 512], output_dim=10, dropout_prob=0.3):
        super(MLP, self).__init__()

        layers = []
        for i in range(len(hidden_layers)):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_layers[i - 1], hidden_layers[i]))
            layers.append(nn.BatchNorm1d(hidden_layers[i]))  # Add BatchNorm layer
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_prob))  # Add Dropout layer
        layers.append(nn.Linear(hidden_layers[-1], output_dim))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.model(x)

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class SGD_SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer=torch.optim.SGD, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        self.base_optimizer = base_optimizer(params, **kwargs)
        self.defaults = self.base_optimizer.defaults
        self.param_groups = self.base_optimizer.param_groups
        self.rho = rho

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = self.rho / (grad_norm + 1e-12)

            for param in group['params']:
                if param.grad is None: continue

                # Apply the SAM perturbation
                e_w = param.grad * scale
                param.add_(e_w)  # climb to the local maximum "w + e(w)"
        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        # Compute the gradient norm before reversing the SAM perturbation.
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            weight_decay = group.get('weight_decay', 0.0)  # Get weight_decay value

            for param in group['params']:
                if param.grad is None: continue

                # Reverse the SAM perturbation
                scale = self.rho / (grad_norm + 1e-12)  # Use the pre-computed grad_norm
                e_w = param.grad * scale
                param.sub_(e_w)  # revert to the original parameters "w - e(w)"

                # Apply weight decay directly to the weights (if nonzero)
                if weight_decay != 0:
                    param.data.add_(param.data, alpha=-weight_decay)

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    # Method to calculate the gradient norm
    def _grad_norm(self):
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2)
                        for group in self.param_groups
                        for p in group['params']
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

    def step(self, closure=None):
        raise RuntimeError('SAM doesn’t implement step() function. Use first_step() and second_step().')

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True)

# Splitting the dataset into training and validation sets if needed
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, _ = torch.utils.data.random_split(train_dataset, [train_size, val_size])


# Define the transformations for the training dataset
train_transforms = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    #v2.RandomCrop(32, padding=4),
    v2.RandomRotation(degrees=15), # Rotates the image by a given angle
    #v2.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), # Randomly changes the brightness, contrast, and other properties
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((28, 28), antialias=True),
    v2.Grayscale(),
    v2.Normalize((0.48075222969055176,), (0.23171930015087128,)),  # Normalizing with calculated mean and std
    Lambda(lambda x: torch.flatten(x))
])

# Define the transformations for the validation dataset (without random transformations)
val_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((28, 28), antialias=True),
    v2.Grayscale(),
    v2.Normalize((0.48075222969055176,), (0.23171930015087128,)),  # Normalizing with calculated mean and std
    Lambda(lambda x: torch.flatten(x))
])

# Custom dataset class that applies the transformations
class TransformDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.dataset)


class CachedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
        self.cache = {}

    def __getitem__(self, idx):
        if idx in self.cache:
            return self.cache[idx]
        else:
            image, label = self.dataset[idx]
            if self.transform:
                image = self.transform(image)
            self.cache[idx] = (image, label)
            return image, label

    def __len__(self):
        return len(self.dataset)

# Apply transformations to the training dataset without caching
train_dataset = TransformDataset(train_dataset, transform=train_transforms)

# Apply transformations to the validation dataset with caching
val_dataset = CachedDataset(TransformDataset(val_dataset, transform=val_transforms))


# Model, criterion, optimizer, and train function remain unchanged (but make sure to use wandb.config for hyperparameters)

def train(model, train_loader, val_loader, criterion, optimizer, epochs, scheduler, use_sam=False, early_stopping_patience=5):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    early_stop = False
    for epoch in range(epochs):
        model.train()
        train_losses = []
        train_accuracies = []
        total_norm = 0

        for batch_idx, (inputs, targets) in enumerate(train_loader):
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            acc = accuracy(outputs, targets)
            train_losses.append(loss.item())
            train_accuracies.append(acc.item())

            optimizer.zero_grad()
            loss.backward()

            # If SAM is used, the first step is to ascent.
            if use_sam:
                optimizer.first_step(zero_grad=False)
                # Make a second forward pass to compute the second gradient
                outputs_second = model(inputs)
                loss_second = criterion(outputs_second, targets)
                loss_second.backward()

                # Log the batch gradients norm before zeroing gradients
                batch_gradients_norm = sum(p.grad.data.norm(2).item() ** 2 for p in model.parameters() if p.grad is not None) ** 0.5
                wandb.log({
                    "batch_training_loss": loss.item(),
                    "batch_gradients_norm": batch_gradients_norm,
                    "batch_idx": epoch * len(train_loader) + batch_idx
                })
                # writer.add_scalar('Loss/Train_batch', loss.item(), epoch * len(train_loader) + batch_idx)
                # writer.add_scalar('Gradients/Norm', batch_gradients_norm, epoch * len(train_loader) + batch_idx)
                optimizer.second_step(zero_grad=True)

            else:

                optimizer.step()
                # Calculate and log batch training loss and gradients norm
                batch_gradients_norm = sum(p.grad.data.norm(2).item() ** 2 for p in model.parameters() if p.grad is not None) ** 0.5

                # Log batch training loss
                wandb.log({
                    "batch_training_loss": loss.item(),
                    "batch_gradients_norm": batch_gradients_norm,
                    "batch_idx": epoch * len(train_loader) + batch_idx
                })

        # Average training loss and accuracy for the epoch
        avg_train_loss = sum(train_losses) / len(train_losses)
        avg_train_accuracy = sum(train_accuracies) / len(train_accuracies)

        # Compute the model norm (L2 norm of the parameters)
        # Compute the total norm of the parameters (for model norm logging).
        total_norm = sum(p.data.norm(2).item() ** 2 for p in model.parameters())
        model_norm = total_norm ** 0.5

        # Perform validation
        model.eval()
        val_losses = []
        val_accuracies = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                acc = accuracy(outputs, targets)
                val_losses.append(loss.item())
                val_accuracies.append(acc.item())

        avg_val_loss = sum(val_losses) / len(val_losses)
        avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)

        # Log epoch metrics to wandb
        wandb.log({
            "epoch": epoch,
            "epoch_training_loss": avg_train_loss,
            "epoch_training_accuracy": avg_train_accuracy,
            "epoch_validation_loss": avg_val_loss,
            "epoch_validation_accuracy": avg_val_accuracy,
            "model_norm": model_norm,
            "learning_rate": scheduler.get_last_lr()[0],
            "batch_size": train_loader.batch_size,
            "optimizer": optimizer.__class__.__name__
        })

        # Update learning rate if scheduler is provided
        if scheduler is not None:
            scheduler.step()

        # Early stopping logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            # Reset the early stopping counter
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            print(f"No improvement in validation loss for {early_stopping_patience} consecutive epochs. Stopping early.")
            early_stop = True
            break

        # Print metrics
        print(f"Epoch {epoch}: avg_train_loss={avg_train_loss}, avg_train_accuracy={avg_train_accuracy}, avg_val_loss={avg_val_loss}, avg_val_accuracy={avg_val_accuracy}")

    return model

# The main function to be called by the sweep
def train_with_wandb():
    # Initialize a new wandb run
    #writer = SummaryWriter(log_dir='runs/my_experiment2')
    wandb.init()

    # Load data
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=wandb.config.batch_size, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=wandb.config.batch_size, shuffle=False, num_workers=0)

    # Initialize model and loss function
    model = MLP()
    criterion = nn.CrossEntropyLoss()

    # Choose the optimizer
    weight_decay = wandb.config.weight_decay  # Get weight decay from wandb.config
    optimizer_name = wandb.config.optimizer
    lr = wandb.config.learning_rate

    # Optimizer-specific hyperparameters
    if optimizer_name == 'sgd' or optimizer_name == 'sgd_sam':
        momentum = wandb.config.get('momentum', 0.9)  # Provide a default value if not specified
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
        if optimizer_name == 'sgd_sam':
            rho = wandb.config.get('rho', 0.05)
            optimizer = SGD_SAM(model.parameters(), base_optimizer=torch.optim.SGD, rho=rho, lr=lr, momentum=momentum, weight_decay= weight_decay)
            use_sam = True
        else:
            use_sam = False
    elif optimizer_name == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        use_sam = False
    elif optimizer_name == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
        use_sam = False
    elif optimizer_name == 'adagrad':
        if 'momentum' in wandb.config:
            print("Warning: Momentum is not used by the Adagrad optimizer and will be ignored.")
        optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay=weight_decay)
        use_sam = False

    # Set up the learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train the model
    train(model, train_loader, val_loader, criterion, optimizer, epochs=30, scheduler=scheduler, use_sam=use_sam, early_stopping_patience=5)

    # Finish the run
    #writer.close()
    wandb.finish()


# Start the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="wandb-final9")
wandb.agent(sweep_id, train_with_wandb)


100%|██████████| 170498071/170498071 [00:02<00:00, 74448098.68it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: bygi5msd
Sweep URL: https://wandb.ai/marius-workspace/wandb-final9/sweeps/bygi5msd


...

wandb: Agent Starting Run: jtf3as10 with config:
wandb: 	batch_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	momentum: 0.9
wandb: 	optimizer: sgd_sam
wandb: 	weight_decay: 0.0001


Epoch 0: avg_train_loss=2.0853281734466553, avg_train_accuracy=0.246425, avg_val_loss=1.9287664746023288, avg_val_accuracy=0.3301154458598726
Epoch 1: avg_train_loss=1.9621783756256104, avg_train_accuracy=0.30075, avg_val_loss=1.8731529978430195, avg_val_accuracy=0.3484275477707006
Epoch 2: avg_train_loss=1.9152925886154175, avg_train_accuracy=0.3188, avg_val_loss=1.848520743619105, avg_val_accuracy=0.3560907643312102
Epoch 3: avg_train_loss=1.8908769289016725, avg_train_accuracy=0.33225, avg_val_loss=1.8285315553094172, avg_val_accuracy=0.3590764331210191
Epoch 4: avg_train_loss=1.8762400741577148, avg_train_accuracy=0.3345, avg_val_loss=1.8147555635233594, avg_val_accuracy=0.36673964968152867
Epoch 5: avg_train_loss=1.8599299215316774, avg_train_accuracy=0.3426, avg_val_loss=1.8087546514098052, avg_val_accuracy=0.3721138535031847
Epoch 6: avg_train_loss=1.8514777139663696, avg_train_accuracy=0.344325, avg_val_loss=1.8026174451135526, avg_val_accuracy=0.37420382165605093
Epoch 7: avg_

batch_gradients_norm,█▇█▆▆▅▅▆▄▄▅▄▅▄▄▄▄▄▄▄▃▃▃▂▃▃▂▃▃▂▂▂▂▁▂▁▂▁▁▁
batch_idx,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_training_loss,█▇█▃▃▄▃▆▄▄▅▄▅▅▄▄▄▃▅▃▃▃▃▂▂▂▁▃▃▂▂▄▅▂▃▄▄▄▄▃
epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
epoch_training_accuracy,▁▅▆▇▇▇▇███████
epoch_training_loss,█▅▃▃▂▂▁▁▁▁▁▂▂▂
epoch_validation_accuracy,▁▄▅▅▇▇█████▇▆▇
epoch_validation_loss,█▅▄▃▂▂▂▁▁▁▂▂▃▃
learning_rate,██████████▁▁▁▁
model_norm,█▇▇▆▅▅▄▄▃▃▂▂▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: epke2rin with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0.9
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.01


Epoch 0: avg_train_loss=2.0906876187324523, avg_train_accuracy=0.2416, avg_val_loss=2.0440503578186036, avg_val_accuracy=0.267
Epoch 1: avg_train_loss=2.0815526272773743, avg_train_accuracy=0.245275, avg_val_loss=2.0205244909286497, avg_val_accuracy=0.2731
Epoch 2: avg_train_loss=2.0743685405731203, avg_train_accuracy=0.2481, avg_val_loss=2.0092275680541993, avg_val_accuracy=0.2876
Epoch 3: avg_train_loss=2.068874038696289, avg_train_accuracy=0.245175, avg_val_loss=1.9913550842285157, avg_val_accuracy=0.2958
Epoch 4: avg_train_loss=2.070186266040802, avg_train_accuracy=0.249025, avg_val_loss=1.9939391542434692, avg_val_accuracy=0.2932
Epoch 5: avg_train_loss=2.0693383916378023, avg_train_accuracy=0.2501, avg_val_loss=1.9910883670806885, avg_val_accuracy=0.3
Epoch 6: avg_train_loss=2.066299139070511, avg_train_accuracy=0.24755, avg_val_loss=1.9958801200866698, avg_val_accuracy=0.2919
Epoch 7: avg_train_loss=2.0653687777996064, avg_train_accuracy=0.2525, avg_val_loss=1.9959843992233277, 